In [38]:
import pandas as pd
import numpy as np

class myVoxel():
    def __init__(self, nx, ny, nz, shortptc, shortptc_legend, alpha_Def1=['number_of_points'], alpha_Def2=['sum','avg'],
                    define_alpha1_as_alpha2type=False):
        self.alpha1 = alpha_Def1
        self.alpha2 = alpha_Def2
        self.nx = nx  
        self.ny = ny
        self.nz = nz
        self.define_alpha1_as_alpha2type = define_alpha1_as_alpha2type
        self.shortptc = shortptc
        self.shortptc_legend = shortptc_legend
        self.Nsamples = self.shortptc.shape[0] 
        self.Nalphas = self.shortptc.shape[1] - 3 
        
        self.all_mins, self.all_maxs = self.create_bb() 
        print('mins and maxs', self.all_mins, self.all_maxs)
        
        self.voxel_size_x, self.voxel_size_y, self.voxel_size_z = self.voxel_sizes() 
        print('voxel sizes', self.voxel_size_x, self.voxel_size_y, self.voxel_size_z)
        
        self.voxel_dict = self.create_voxel()
        #print('voxel_dict', self.voxel_dict)
        
        self.voxel_matrix = self.create_voxel_matrix() 
        print('voxel_matrix', self.voxel_matrix.shape)
        #print('voxel_matrix', self.voxel_matrix)
        
        del self.shortptc
        del self.voxel_dict
    
    def create_bb(self,): 
        all_mins = [np.min(self.shortptc[k]) for k in ['x','y','z']] 
        all_maxs = [np.max(self.shortptc[k]) for k in ['x','y','z']]  
        return all_mins, all_maxs
    
    def voxel_sizes(self,):
        scales = [self.all_maxs[k] - self.all_mins[k] for k in range(3)]
        for k in range(3): 
            voxel_size_x = scales[0] / self.nx 
            voxel_size_y = scales[1] / self.ny
            voxel_size_z = scales[2] / self.nz
        return voxel_size_x, voxel_size_y, voxel_size_z
        
    def create_voxel(self,):
        voxel_dict = {} 
        
        for ix in range(self.nx): 
            voxel_dict[ix] = {}            
            for iy in range(self.ny):
                voxel_dict[ix][iy] = {}                
                for iz in range(self.nz):
                    voxel_dict[ix][iy][iz] = {'list_of_nodes': [],  
                                              'prop1':[],
                                              'prop2':[]}
                    
        for pt in range(self.shortptc.shape[0]): 
            var = (self.shortptc['x'][pt] - self.all_mins[0]) /self.voxel_size_x # Why all_mins[0]? first column?
            #print(var)
            voxel_ix = int(np.floor(var)) 
            #print(voxel_ix)
            
            var = (self.shortptc['y'][pt] - self.all_mins[1]) / self.voxel_size_y 
            #print(var)
            voxel_iy = int(np.floor(var))
            #print(voxel_iy)
            
            var = (self.shortptc['z'][pt] - self.all_mins[2]) / self.voxel_size_z
            #print(var)
            voxel_iz = int(np.floor(var))
            #print(voxel_iz)
            
            if voxel_ix >= self.nx:
                voxel_ix = self.nx-1
            if voxel_iy >= self.ny:
                voxel_iy = self.ny-1
            if voxel_iz>=self.nz:
                voxel_iz = self.nz-1
            
            voxel_dict[voxel_ix][voxel_iy][voxel_iz]['list_of_nodes'].append(pt)
     
        
        for ix in range(self.nx):
            for iy in range(self.ny):
                for iz in range(self.nz):
                    for prop in self.alpha1:
                        if prop=='number_of_points':
                            voxel_dict[ix][iy][iz]['prop1'].append(len(voxel_dict[ix][iy][iz]['list_of_nodes']))
                    if len(voxel_dict[ix][iy][iz]['list_of_nodes'])>0:
                        for p, prop in enumerate(self.alpha2):
                            if prop=='sum':
                                voxel_dict[ix][iy][iz]['prop2'].append(np.sum(np.take(self.shortptc[self.shortptc_legend[p]],
                                                                                 indices=voxel_dict[ix][iy][iz]['list_of_nodes'], 
                                                                                 axis=0)))
                            elif prop=='avg':
                                voxel_dict[ix][iy][iz]['prop2'].append(np.mean(np.take(self.shortptc[self.shortptc_legend[p]],
                                                                                 indices=voxel_dict[ix][iy][iz]['list_of_nodes'], 
                                                                                 axis=0)))
                            else:
                                print('unkoen prop:', prop)
                    else:
                        voxel_dict[ix][iy][iz]['prop2'] = [0. for k in range(self.Nalphas)]
                    #print('my vox', ix,iy,iz,voxel_dict[ix][iy][iz]['prop2'])
                                 
        return voxel_dict
        
    def create_voxel_matrix(self, ):
        if self.define_alpha1_as_alpha2type:
            voxelmatrix = np.empty((self.nx, self.ny, self.nz, self.Nalphas+len(self.alpha1)))
            for ix in range(self.nx):
                for iy in range(self.ny):
                    for iz in range(self.nz):
                        voxelmatrix[ix, iy, iz, :] = np.concatenate((self.voxel_dict[ix][iy][iz]['prop2'],
                                                                     self.voxel_dict[ix][iy][iz]['prop1']), axis=0)
        else:
            voxelmatrix = np.empty((self.nx, self.ny, self.nz, self.Nalphas))
            for ix in range(self.nx):
                for iy in range(self.ny):
                    for iz in range(self.nz):
                        #print(self.voxel_dict[ix][iy][iz]['prop2'])
                        voxelmatrix[ix, iy, iz, :] = self.voxel_dict[ix][iy][iz]['prop2']
        return voxelmatrix

    

In [39]:
# Read in the Data and Pre-process
df = pd.read_csv("Preproc/shortptc.csv", sep=', |,', header=0, engine='python')
df['tightness'] = df[["mpc", "rigid", "fastener"]].sum(axis=1)

all_tie_names = np.array([k for k in df.columns if 'tie_' in k])
all_tie_names_float = np.array([float(k.split('_')[1]) for k in all_tie_names])
all_ties_smaller_than_5 = np.where(all_tie_names_float<=5)[0]
all_ties_bigger_than_5 = np.where(all_tie_names_float>5)[0]

df['ties_under_5'] = df[all_tie_names[all_ties_smaller_than_5]].sum(axis=1)
df['ties_equal_over_5'] = df[all_tie_names[all_ties_bigger_than_5]].sum(axis=1)

In [40]:
my_column_names = df.columns
column_names_that_contain_kwd = [k for k in my_column_names if 'conn_sec' in k]
df['connectors'] = df[column_names_that_contain_kwd].sum(axis=1)

In [41]:
shortptc = df.drop(labels=['index_x','index_y','index_z']+column_names_that_contain_kwd+["mpc", "rigid", "fastener"]+list(all_tie_names), axis=1)
print(shortptc.shape)

(99, 15)


In [42]:
shortptc_legend = shortptc.columns
print(shortptc_legend, len(shortptc_legend))

Index(['x', 'y', 'z', 'mass+nsm', 'bulk', 'rp02', 'bc_displacement_1_6_0.0',
       'ic_stress_0.00023654_0.001_0.0', 'conn_load_1.0_-40100.0',
       'conn_load_1.0_53100.0', 'conn_load_1.0_-64100.0', 'tightness',
       'ties_under_5', 'ties_equal_over_5', 'connectors'],
      dtype='object') 15


In [68]:
nx = 2
ny = 2
nz = 2
alpha1 = ['number_of_points']
alpha2 = ['avg', 'avg', 'avg', 'avg', 'avg', 'avg', 'avg', 'avg', 'sum', 'sum', 'sum', 'sum' ]
print(len(alpha2))

12


In [73]:
train_voxel = myVoxel(nx, ny, nz, shortptc, shortptc_legend, alpha_Def1=alpha1, alpha_Def2=alpha2, define_alpha1_as_alpha2type=False).voxel_matrix

mins and maxs [-860.3588679999999, -528.369141, -105.15775] [-831.179387, -436.27149800000007, 125.81103999999999]
voxel sizes 14.589740499999948 46.048821499999974 115.48439499999999
voxel_matrix (2, 2, 2, 12)


In [74]:
test_voxel = myVoxel(nx, ny, nz, shortptc, shortptc_legend, alpha_Def1=alpha1, alpha_Def2=alpha2, define_alpha1_as_alpha2type=False).voxel_matrix

mins and maxs [-860.3588679999999, -528.369141, -105.15775] [-831.179387, -436.27149800000007, 125.81103999999999]
voxel sizes 14.589740499999948 46.048821499999974 115.48439499999999
voxel_matrix (2, 2, 2, 12)


In [77]:
# preprocessinbg -> use only training to calcualte the mina nd amx 
# calcaulate scaling variables using training data
new_max = 1.0
new_min = 0.
max_alphas = np.max(train_voxel, axis=-1) # nx xy xz 
min_alphas = np.min(train_voxel, axis=-1) # nx xy xz 
#print('max_alphas',max_alphas)
#print('min_alphas',min_alphas)
# preprocess all the samples 
def preprocess_voxel(train_voxel, max_alphas, min_alphas, new_max, new_min):
    before_max_alphas = np.max(train_voxel, axis=-1) # nx xy xz 
    before_min_alphas = np.min(train_voxel, axis=-1) # nx xy xz 
    #print('before_max_alphas',before_max_alphas)
    #print('before_min_alphas',before_min_alphas)
    denominator = max_alphas-min_alphas # nx xy xz 
    inds = denominator>1e-10  # nx xy xz 
    tmp_max = np.reshape(max_alphas, list(inds.shape)+[1])
    tmp_min = np.reshape(min_alphas, list(inds.shape)+[1])
    train_voxel[inds] = ((train_voxel[inds]-tmp_min[inds])*new_max + (tmp_max[inds]-train_voxel[inds])*new_min)/(tmp_max[inds]-tmp_min[inds])
    after_max_alphas = np.max(train_voxel, axis=-1) # nx xy xz 
    after_min_alphas = np.min(train_voxel, axis=-1) # nx xy xz 
    #print('after_max_alphas',after_max_alphas)
    #print('after_min_alphas',after_min_alphas)
    return train_voxel
train_voxel = preprocess_voxel(train_voxel, max_alphas, min_alphas, new_max, new_min)
test_voxel = preprocess_voxel(test_voxel, max_alphas, min_alphas, new_max, new_min)